# PetroChron

Figures for paper, and a few extra

In [ ]:
import numpy as np

import pandas as pd
pd.set_option("display.max_columns", None)

import imageio

# pip install agrid
from agrid.grid import Grid
import cartopy.crs as ccrs

from matplotlib import pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import PercentFormatter,  ScalarFormatter

from scipy.signal import argrelmax
from scipy import stats

from cmcrameri import cm

km = 1000

In [ ]:
# import some pretty background extent=[-3000*km, 3000*km, -3000*km, 3000*km]

# Background hill hill shade from CryoSat-2 and RAMP-2
# Rock outcrops from Burton-Johnson et al 2016
# Done using Quantarctica 
    
im = imageio.imread('data/background.png')
background = np.asarray(im)[::-1, :, :] # Flip y ax from row id to increasing y

plt.imshow(background, origin='lower')

In [ ]:
# Import latest version of PetroChron
# e.g.:
#! wget https://zenodo.org/record/5032026/files/PetroChron_Antarctica_db_0_1.csv

f_name = '../data/petrochron/PetroChron_Antarctica_db_0_1.csv'
df = pd.read_csv(f_name, index_col=False)

In [ ]:
# Set up a continental agrid object
ant = Grid(crs='epsg:3031', res = [50*km, 50*km], extent=[-3000*km, 3000*km, -3000*km, 3000*km])

In [ ]:
# Some default values

# marker size for all plots
ms = 250

# https://colorbrewer2.org/#type=qualitative&scheme=Paired&n=10 (I think)
fig_colors = ['#8dd3c7','#ffffb3','#bebada','#fb8072',
              '#80b1d3','#fdb462','#b3de69','#fccde5',
              '#d9d9d9','#bc80bd']

# increase resolution, lower this to save memory and speed up 
plt.rcParams['figure.dpi'] = 300

In [ ]:
ax = None

def scatter_selection_map(mask, 
                df = df, 
                ms=ms, 
                label = '',
                alpha= 1,
                marker = 'o',
                linewidths = 0.9, 
                zorder = 100,
                c = None,
                map_crs = ccrs.PlateCarree(),
                edgecolors = 'k',
                ax=ax):
    '''
    Macro to plot cathegorical data
    '''
    
    lon = df['longitude'][mask]
    lat = df['latitude'][mask]
    
    return ax.scatter(lon, lat, 
                c = c,
                s = ms,
                label = label,
                linewidths = linewidths,
                edgecolors = edgecolors,
                marker = marker,
                zorder = zorder, 
                alpha = alpha,
                transform=map_crs, )
  
    
def scatter_data_map(data_col, 
                     df=df, 
                     remove_empty = True,
                     marker = 'o',
                     sort = False, 
                     linewidths = 0.9,
                     ascending=True, 
                     cmap = 'viridis', 
                     vmin = None,
                     vmax = None,
                     label = '',
                     ms=ms,
                     zorder = 100,
                     map_crs = ccrs.PlateCarree(),
                     ax=ax):
    '''
    Macro to plot data with values
    '''


    df_plot = df.copy()

    if sort:
        df_plot = df_plot.sort_values(data_col, ascending = ascending)

    if remove_empty:
        df_plot = df_plot[df_plot[data_col].notna()]

    return ax.scatter(df_plot['longitude'], 
                      df_plot['latitude'], 
            c = df_plot[data_col],
            cmap = cmap,
            s = ms,
            label = label,
            vmin = vmin,
            vmax = vmax,
            linewidths = linewidths,
            edgecolors = 'k',
            marker = marker,
            zorder = zorder, 
            alpha = 1,
            transform=map_crs, )

   
    
    
def export_legend(handles, labels, 
                  title = '', 
                  y_label = '', 
                  ncol = 1, 
                  markerscale = 0.65,
                  ms = ms,
                  filename="legend.pdf"):
    '''
    Save legend as searate file
    '''
    
    plt.show()
    
    
    legend = plt.legend(handles, 
                        labels, 
                        ncol = ncol,
                        loc=3, 
                        markerscale = markerscale,
                        handletextpad=0,
                        framealpha=1, 
                        title = title, 
                        title_fontsize=12, 
                        frameon=False, 
                        prop={'size': 11})
    
    legend._legend_box.align = 'left'
    

    fig  = legend.figure
    plt.xticks([])
    plt.yticks([])
    plt.setp(plt.gca().spines.values(), visible=False) 
    fig.canvas.draw()
    bbox  = legend.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    fig.savefig(filename, dpi="figure", bbox_inches=bbox, transparent=True)
    return None

    
def make_legend(values, 
                file_name = 'legend.pdf', 
                cmap = 'viridis', 
                ncol = 1, 
                title = '', 
                s=ms, 
                markerscale = 0.65,
                vmin=None, 
                vmax=None, 
                linewidths=0.9):

    try:
        plt.show()
    except:
        pass
    
    fig, ax = plt.subplots(1,1, figsize = (16,16))

    for v in values:
        norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)
    
        ax.scatter([v],[v], 
               color = mpl.cm.get_cmap(cmap)(norm(v)), 
               label = v, 
               s=s, 
               linewidths=linewidths, 
               edgecolors='k')
    

    handles, labels = ax.get_legend_handles_labels()
    plt.close()


    legend = plt.legend(handles, 
                        labels, 
                        ncol = ncol,
                        loc=3, 
                        framealpha=1, 
                        markerscale = markerscale,
                        title = title, 
                        title_fontsize=12, 
                        handletextpad=0,
                        frameon=False, 
                        prop={'size': 11})
    
    fig  = legend.figure
    plt.xticks([])
    plt.yticks([])
    plt.setp(plt.gca().spines.values(), visible=False) 
    fig.canvas.draw()
    legend._legend_box.align = 'left'
    bbox  = legend.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    fig.savefig(file_name, dpi="figure", bbox_inches='tight', transparent=True)
    return None
    
 

In [ ]:
# Fig 1a

fig, ax = ant.map_grid(background, return_fig_ax=True, coastline_res='10m', figsize=(16,16)) 

is_both = np.logical_and(df['geochro_id'] > 0, df['geochem_id'] >0)
is_neither = ~np.logical_or(df['geochro_id'] > 0, df['geochem_id'] >0)

is_geochem = df['geochem_id'] > 0
is_geochron = df['geochro_id'] > 0

geochem = scatter_selection_map(is_geochem, ax=ax, c = fig_colors[0])     
geochron = scatter_selection_map(is_geochron, ax=ax, c = fig_colors[1])   
both = scatter_selection_map(is_both, ax=ax, c = fig_colors[2])

fig.savefig('fig/fig_1a.pdf')


export_legend([geochem, geochron, both], 
              ['geochemistry (N=%s)'%sum(is_geochem), 
               'geochronology (N=%s)'%sum(is_geochron), 
               'geochemistry and geochronology (N=%s)'%sum(is_both), ],
               filename='fig/fig_1a_legend.pdf')




In [ ]:
# A few entries are empty
print(df[is_neither]['sample_name'])

In [ ]:
# Fig 1b

fig, ax = ant.map_grid(background, return_fig_ax=True, coastline_res='10m',figsize=(16,16))


is_bas = df['data_source']=='BAS_GEOLDB'
bas = scatter_selection_map(is_bas, ax=ax, c = fig_colors[3])  

is_ozchem = df['data_source']=='OZCHEM'
ozchem = scatter_selection_map(is_ozchem, ax=ax, c = fig_colors[4])  

is_petlab = df['data_source']=='Petlab'
petlab = scatter_selection_map(is_petlab, ax=ax, c = fig_colors[5])   


fig.savefig('fig/fig_1b.pdf')

export_legend([petlab, ozchem, bas], 
              ['Petlab (N=%s)'%sum(is_petlab),
               'OZCHEM (N=%s)'%sum(is_ozchem),
               'BAS geological database (N=%s)'%sum(is_bas),],
               filename='fig/fig_1b_legend.pdf')



In [ ]:
# Fig 1c
fig, ax = ant.map_grid(background, return_fig_ax=True, coastline_res='10m', figsize=(16,16))


is_georoc = df['data_source']=='GEOROC'
georoc = scatter_selection_map(is_georoc, ax=ax, c = fig_colors[6])     

is_dateview = df['data_source']=='DateView'
dateview = scatter_selection_map(is_dateview, ax=ax, c = fig_colors[7])      

fig.savefig('fig/fig_1c.pdf')

export_legend([georoc, dateview, ], 
              [ 'GEOROC (N=%s)'%sum(is_georoc), 
               'DateView (N=%s)'%sum(is_dateview)],
               filename='fig/fig_1c_legend.pdf')



In [ ]:
fig, ax = ant.map_grid(background, return_fig_ax=True, coastline_res='10m', figsize=(16,16))

used_sources = ['DateView', 'GEOROC', 'Petlab', 'OZCHEM', 'BAS_GEOLDB']
is_other = ~df['data_source'].isin(used_sources)


others = scatter_selection_map(is_other, ax=ax, c = fig_colors[9])    
  

fig.savefig('fig/fig_1d.pdf')


export_legend([others], 
              ['other sources \n(published, unpublished) (N=%s)'%sum(is_other)],
               filename='fig/fig_1d_legend.pdf')



In [ ]:
rock_types = ['clastic',
 'plutonic',
 'volcanic',
 'not defined, igneous',
 'metaplutonic',
 'metavolcanic',              
 'metaigneous',
 'metasedimentary',
 'not defined, metamorphic',
 'not defined']


rock_labels = [
 'sedimentary\n(clastic, biogenic)',
 'plutonic',
 'volcanic',
 'igneous\nundifferentiated',
 'metaplutonic',
 'metavolcanic',
 'metaigneous',
 'metasedimentary',
 'metamorphic\nundifferentiated',
 'unclassified',]


rock_labels_short = [
 'sedimentary',
 'plutonic',
 'volcanic',
 'igneous (undiff.)',
 'metaplutonic',
 'metavolcanic',
 'metaigneous',
 'metasedimentary',
 'metamorphic (undiff.)',
 'unclassified',]



sedimentary_colors = ['#ffed6f']
igneous_colors = ['#fee8c8','#fdbb84','#e34a33'][::-1]
metamorphic_colors = ['#e1f2f2','#b2e2e2','#66c2a4','#2ca25f','#006d2c'][::-1]

undefined_color = ['gray']

marker_colors = sedimentary_colors + igneous_colors + metamorphic_colors + undefined_color
                 

fig, ax = ant.map_grid(background, return_fig_ax=True, coastline_res='10m', figsize=(16,16)) # extent = tam.extent


rock_type_handles = []
rock_type_n = []

for rock_type, label, color  in zip(rock_types, rock_labels, marker_colors):
    if rock_type not in ['biogenic','clastic']:
        is_rock_type = df['rock_type']==rock_type
    else:
        is_rock_type = df['rock_type'].isin(['biogenic','clastic'])
        
    if rock_type == 'not defined':
        zorder = 1
    else:
        zorder = 20
        
    rock_type_handles.append(scatter_selection_map(is_rock_type, 
                                                   ax=ax, 
                                                   c= color, 
                                                   zorder = zorder) )
    rock_type_n.append(sum(is_rock_type))
    print(rock_type, sum(is_rock_type))
    

fig.savefig('fig/fig_3a.pdf')
    

    
empty = ax.scatter([],[], c='w', alpha = 0)


for gap in [1, 10, 11, 12, 13]:
    rock_type_handles.insert(gap, empty)
    rock_labels_short.insert(gap, '')
   
export_legend(rock_type_handles, rock_labels_short, ncol=3,
               filename='fig/fig_3a_legend.pdf')


In [ ]:
fig, ax = plt.subplots(1,1, figsize = (5,7))

height = 0.6

ind = -np.arange(len(rock_type_n))*0.7

ax.vlines([0], ymin = ind.min()-height, ymax=ind.max()+height, color='gray', alpha =0.5)  
 

ax.barh(ind, 
        rock_type_n, 
        height = height,
        color = marker_colors,
        edgecolor = 'gray',
        align='center')

ax.set_yticks(ind)
ax.set_yticklabels([])
ax.set_xticklabels([])
ax.set_xticks([])
ax.set_frame_on(False)



   

for n, i, rock_type, label, color  in zip(rock_type_n, ind, rock_types, rock_labels, marker_colors):
    ax.text(n + 80, i, 'N = %s'%str(n), color='k', fontsize=11, ha='left', va='center')    
    ax.text(- 100, i,label, color='k', fontsize=11, ha='right', va='center')
    

fig.savefig('fig/fig_3b.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (5,5))

labels, counts = np.unique(df['rock_group'], return_counts=True)

colors = [igneous_colors[1]] + [metamorphic_colors[2]] + undefined_color + sedimentary_colors



print(labels, counts)


_, texts, autotexts = ax.pie(counts, labels=labels,
       autopct='%1.1f%%', startangle=90, 
       colors = colors, 
        wedgeprops={"edgecolor":"k",'linewidth': 0.5, 'antialiased': True},
       pctdistance=0.6, labeldistance=1.2)
ax.axis('equal')


texts[0]._x = + 0.8

texts[1]._y =- 0.3
texts[1]._x =- 0.63

texts[2]._x =+ 0.4
texts[3]._x =- 0.4

texts[2]._y - 0.1
#texts[3]._y = texts[3]._y - 0

autotexts[1]._y = autotexts[1]._y 
autotexts[2]._x = autotexts[2]._x + 0.2
autotexts[3]._x =- 0.24

autotexts[2]._y =+ 0.8
autotexts[3]._y =+ 0.8


for t in texts:
    t.set_horizontalalignment('center')
    
fig.savefig('fig/fig_3b_pie.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
fig, ax = ant.map_grid(background, return_fig_ax=True, coastline_res='10m', figsize=(16,16))


scatter = scatter_data_map('sio2', 
                     sort = False, 
                     ascending=True, 
                     cmap = cm.roma_r, 
                     vmin = 25,
                     vmax = 75,
                     ms = ms,
                     ax=ax)


fig.savefig('fig/fig_3c.pdf')
    

In [ ]:
make_legend([30, 40, 50, 60, 70, 80], 
            'fig/fig_3c_legend.pdf', 
            title = 'wt. %',
            cmap=cm.roma_r, 
            vmin = 25, vmax = 75)

In [ ]:
# import global database

gard_f = '../data/rock_database/sample.csv'
gard = pd.read_csv(gard_f, low_memory=False)


In [ ]:
gard = gard.merge(pd.read_csv('../data/rock_database/country.csv'), on='country_id', suffixes='')
gard = gard.merge(pd.read_csv('../data/rock_database/major.csv'), on='major_id', suffixes='')
# gard = gard.merge(pd.read_csv('../data/rock_database/method.csv'), on='method_id', suffixes='')
# gard = gard.merge(pd.read_csv('../data/rock_database/isotope.csv'), on='iso_id', suffixes='')
gard = gard.merge(pd.read_csv('../data/rock_database/rockgroup.csv'), on='rgroup_id', suffixes='')
# gard = gard.merge(pd.read_csv('../data/rock_database/age.csv'), on='age_id', suffixes='')
gard = gard.merge(pd.read_csv('../data/rock_database/computed.csv'), on='comp_id', suffixes='')



In [ ]:
ocean_mask = gard['country_id']!=152

fig, ax = plt.subplots(1,1, subplot_kw = {'projection': ccrs.Mollweide()})

ax.set_global()
ax.coastlines()
ax.gridlines()
    
ax.scatter(gard.longitude[ocean_mask], gard.latitude[ocean_mask ], 
            s = 0.6, transform=ccrs.PlateCarree(), label = 'samples used')


ax.scatter(gard.longitude[~ocean_mask], gard.latitude[~ocean_mask ], 
            s = 0.6,  transform=ccrs.PlateCarree(), label = 'exluded ocean')



ax.legend(loc=(0.9,0.8), handletextpad=0)

In [ ]:
rock_group_colors = [igneous_colors[1]] + [metamorphic_colors[2]] + sedimentary_colors
rock_groups = ['igneous', 'metamorphic', 'sedimentary']

In [ ]:
fig, ax = plt.subplots(4,1, figsize = (6,6), sharex=True)

width = 4.1
si_bins = np.array([0, 45, 55, 65, 100])
x_pos = np.array([40, 50, 60, 70])

x_min = 35

for rock_group_color, rock_group, a in zip(rock_group_colors + ['w'], rock_groups + ['All'], ax):

    a.set_ylabel(rock_group, 
                 labelpad=12,
                 fontsize=11, 
                 bbox=dict(facecolor=rock_group_color, 
                                       alpha = 0.5,
                                       edgecolor='none', 
                                       boxstyle='round,pad=0.25'))

    if rock_group is not 'All':
         pc_si = np.histogram(df['sio2'][ df['rock_group']==rock_group].dropna(), 
                     bins=si_bins, 
                     range=(0, 100))[0]
            
         gard_si = np.histogram(gard['sio2'][ocean_mask][ gard['rock_group']==rock_group].dropna(), 
                       bins=si_bins, 
                       range=(0, 100))[0]
        
        
    else:
         pc_si = np.histogram(df['sio2'].dropna(), 
                     bins=si_bins, 
                     range=(0, 100))[0]
            
         gard_si = np.histogram(gard['sio2'][ocean_mask].dropna(), 
                       bins=si_bins, 
                       range=(0, 100))[0]
        

    pc_si = pc_si / np.sum(pc_si)
    gard_si = gard_si / np.sum(gard_si)
    
    a.bar(x_pos-width/1.8, gard_si, width=width, label='global compilation', color='#a6cee3')
    a.bar(x_pos+width/1.8, pc_si, width = width, label = 'PetroChron', color='#b2df8a')
    
    
    for x, y in zip(x_pos-width/1.8, gard_si):
        a.text(x, y+0.01, '%.1f%%'%(y*100),
               fontsize=11, 
               ha='center')

    for x, y in zip(x_pos+width/1.8, pc_si):
        a.text(x, y+0.01, '%.1f%%'%(y*100), 
               fontsize=11,
               ha='center')
        
        
    a.yaxis.set_major_formatter(PercentFormatter(1))
    a.set_xlim(x_min,75)
    a.set_ylim(0,0.50)
    a.spines['right'].set_visible(False)
    a.spines['top'].set_visible(False)
    

ax[-1].set_ylabel('all')
ax[-1].set_xticklabels(['ultramafic\n<45% wt', 
                    'mafic\n45-55% wt', 
                    'intermediate\n55-65% wt', 
                    'felsic\n>65% wt'], 
                    fontsize=11,
                   rotation=0)

ax[-1].set_xticks([40, 50, 60, 70])
ax[0].legend(loc = (0.6,0.9), fontsize=11,)

fig.savefig('fig/fig_3d.pdf')

In [ ]:
colors_df = pd.read_json('data/ChronostratChart2018-08_rgb.json', lines=True)
ages_df = pd.read_json('data/ChronostratChart2018-08.json', lines=True)

labels = ages_df['Era']

# Set periods in Phanerozoic
labels[24:-16] = ages_df['Period'][24:-16]

colors = colors_df['Era']
colors[24:-16] = colors_df['Period'][24:-16]
 
    
cols = list(colors)
ags = [0] + list(ages_df['Start']) 
    
  
for i, col in enumerate(cols):
        if col is None:
            cols[i] = [0.,0.,0.,0.]
        else:
            cols[i] = col.split('/') + [255]
            cols[i] = [float(c)/255 for c in cols[i]]

    
cmap_strat, norm_strat = mpl.colors.from_levels_and_colors(ags, cols)
 
    
fig, ax = plt.subplots(1,1, figsize = (3,16))

xs = [0]*len(norm_strat.boundaries)
ys = np.linspace(0,1,len(norm_strat.boundaries))


ax.set_xticks([])
ax.set_yticks([])
ax.scatter(xs, -ys, c=norm_strat.boundaries, norm = norm_strat, cmap = cmap_strat)
ax.set_frame_on(False)


for x, y, age, label in zip(xs[::2], ys[::2], norm_strat.boundaries[::2], labels[::2]):
    ax.text(x+0.01, -y, str(age) + ' ' + str(label))

    

legend_cols, legend_i = np.unique(cols, axis=0, return_index=True)

df_chron_legend = pd.DataFrame(
    {'index':legend_i, 'colors':legend_cols.tolist(), 'labels':labels[legend_i]}).sort_index()



In [ ]:
fig, ax = plt.subplots(1,1)


for i, row in df_chron_legend.iterrows():
    ax.scatter(i,i, color = row['colors'], label = row['labels'], s=ms, 
               linewidths=0.9, edgecolors='k')
    

handles, labels = ax.get_legend_handles_labels()
plt.close()


legend = plt.legend(handles, 
                        labels, 
                        ncol = 1,
                        loc=3, 
                        framealpha=1, 
                        handletextpad=0,
                        title = 'Stratigraphic age', 
                        title_fontsize=12, 
                        frameon=False, 
                        markerscale = 0.65,
                        prop={'size': 11})
fig  = legend.figure
plt.xticks([])
plt.yticks([])
plt.setp(plt.gca().spines.values(), visible=False) 
fig.canvas.draw()
legend._legend_box.align = 'left'
bbox  = legend.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
fig.savefig('fig/fig_4a_legend.pdf', dpi="figure", bbox_inches='tight', transparent=True)
    
    


In [ ]:
# Manually select isotopes to display

cooling_isotopes = ['amphibole', 
'apatite',
'biotite',
'biotite - whole rock',
'chlorite',
'feldspar - plagioclase',
'feldspar - potassic',
'garnet - whole rock',
'glass',
'hornblende',
'minerals, various',
'muscovite',
'muscovite (sericite)',
'rutile',
'titanite',
'whole rock',
'whole rock + various minerals',
'zircon',]


metamorphic_isotopes = ['zircon', 'garnet',  'garnet - whole rock', 'monazite', ]

In [ ]:
used_geochron_mask = df['age_significance'].isin(['Metamorphism (contact)', 
                                                  'Metamorphism (regional)',
                                                  'Crystallisation (intrusive)', 
                                                  'Crystallisation (extrusive)',
                                                 'Cooling'])

chron_types = sorted([_ for _ in list(set(df['mineral_isotope'][used_geochron_mask])) if str(_) != 'nan'])

chron_markers = [(2,1,0), 
                 (2,1,90), 
                 (3,0,0), 
                 (3,0,90), 
                 (3,0,180), 
                 (3,0,270),
                 (4,0,0), 
                 (4,0,45),
                 (4,1,0),
                 (4,1,45),
                 (5,0,0), 
                 (5,1,0),
                 (5,0,36), 
                 (5,1,36),
                 (6,0,0), 
                 (6,1,0),
                 (6,0,30), 
                 (6,1,30),
                 (7,0,0),
                 (7,1,0),
                 (8,0,45), 
                 (8,1,45),
                 'P', 
                'o',
                '.', ]


chron_dict = dict(zip(chron_types, chron_markers))

fig_legend, ax_legend = plt.subplots(1,1)
for chron_type in sorted(list(set(cooling_isotopes+metamorphic_isotopes))):

    ax_legend.scatter([100], 
                [100], 
                c = 'white',
                s = ms,
                label = '%s'%chron_type,
                linewidths = 0.9,
                edgecolors = 'k',
                marker = chron_dict[chron_type],
                zorder = 20, 
                alpha = 1) 
    ax_legend.set_xlim(0,0.001)
    ax_legend.set_ylim(0,0.001)
    


handles, labels = ax_legend.get_legend_handles_labels()


legend = plt.legend(handles, 
                        labels, 
                        ncol = 1,
                        loc=3, 
                        handletextpad=0,
                        markerscale = 0.9,
                        framealpha=1, 
                        title = 'Mineral isotope', 
                        title_fontsize=12, 
                        frameon=False, 
                        prop={'size': 11})

legend._legend_box.align = 'left'
fig  = legend.figure
plt.xticks([])
plt.yticks([])
plt.setp(plt.gca().spines.values(), visible=False) 
fig.canvas.draw()
legend._legend_box.align = 'left'
bbox  = legend.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
fig.savefig('fig/fig_4c_legend.pdf', dpi="figure", bbox_inches='tight', transparent=True)

In [ ]:
# Mark inset extents  
tam_extent = [-350*km, 700*km, -250*km, -1500*km]
pb_extent = [1550*km, 2400*km, 350*km, 1000*km]

x_vert_pb = [pb_extent[0], pb_extent[1], pb_extent[1], pb_extent[0], pb_extent[0]]
y_vert_pb = [pb_extent[3], pb_extent[3], pb_extent[2], pb_extent[2], pb_extent[3]]

x_vert_tam = [tam_extent[0], tam_extent[1], tam_extent[1], tam_extent[0], tam_extent[0]]
y_vert_tam = [tam_extent[3], tam_extent[3], tam_extent[2], tam_extent[2], tam_extent[3]]

In [ ]:
fig, ax = ant.map_grid(background, return_fig_ax=True, coastline_res='10m', figsize=(16,16)) # extent = tam.extent


ax.plot(x_vert_pb, y_vert_pb, transform=ccrs.SouthPolarStereo(), 
        zorder =20, c = 'k', alpha = 0.7, linestyle='--')



ax.plot(x_vert_tam,y_vert_tam, transform=ccrs.SouthPolarStereo(), 
        zorder =20, c = 'k', alpha = 0.7, linestyle='--')


wsp = 7.5

pb_figsize = (wsp, wsp*np.abs(pb_extent[3]-pb_extent[2])/np.abs(pb_extent[1]-pb_extent[0]))
tam_figsize = (wsp, wsp*np.abs(tam_extent[3]-tam_extent[2])/np.abs(tam_extent[1]-tam_extent[0]))


fig_pb, ax_pb = ant.map_grid(background, 
                       return_fig_ax=True, 
                       coastline_res='10m',  
                       extent=pb_extent, 
                        figsize=pb_figsize
                       )


fig_tam, ax_tam = ant.map_grid(background, 
                       return_fig_ax=True, 
                       coastline_res='10m',  
                       extent=tam_extent, 
                        figsize=tam_figsize,
                       )


crystalisation_mask = (df['age_significance'].isin(['Crystallisation (intrusive)', 
                                                      'Crystallisation (extrusive)'])) & (df['mineral_isotope'] == 'zircon') 

for a in [ax, ax_pb, ax_tam]:
    a.scatter(df['longitude'][crystalisation_mask], 
            df['latitude'][crystalisation_mask], 
            c = df['age_ma'][crystalisation_mask],
            cmap = cmap_strat,
            norm = norm_strat,
            s = ms,
            label = chron_type,
            linewidths = 0.9,
            edgecolors = 'k',
            marker = 'o',
            zorder = 20, 
            alpha = 1,
            transform=ccrs.PlateCarree())
    
    
fig_pb.savefig('fig/fig_4a_pb.pdf', bbox_inches='tight', pad_inches=0)

fig_tam.savefig('fig/fig_4a_tam.pdf', bbox_inches='tight', pad_inches=0)

fig.savefig('fig/fig_4a.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
fig, ax = ant.map_grid(background, return_fig_ax=True, coastline_res='10m', figsize = (16,16))

ax.plot(x_vert_pb, y_vert_pb, transform=ccrs.SouthPolarStereo(), 
        zorder =20, c = 'k', alpha = 0.7, linestyle='--')  

fig_pb, ax_pb = ant.map_grid(background, 
                       return_fig_ax=True, 
                       coastline_res='10m',  
                       extent=pb_extent, 
                        figsize=pb_figsize
                       )




metamorphic_mask = df['age_significance'].isin(['Metamorphism (contact)', 'Metamorphism (regional)'])
for chron_type in metamorphic_isotopes:
    chron_mask = df['mineral_isotope'] == chron_type
    mask = np.logical_and(chron_mask, metamorphic_mask)
    
    for a in [ax, ax_pb]:
        a.scatter(df['longitude'][mask], 
                df['latitude'][mask], 
                c = df['age_ma'][mask],
               cmap = cmap_strat,
                norm = norm_strat,
                s = ms,
                label = label,
                linewidths = 0.9,
                edgecolors = 'k',
                marker = chron_dict[chron_type],
                zorder = 20, 
                alpha = 1,
                transform=ccrs.PlateCarree(), )

    

fig_pb.savefig('fig/fig_4b_pb.pdf', bbox_inches='tight', pad_inches=0)

fig.savefig('fig/fig_4b.pdf', bbox_inches='tight', pad_inches=0)
    

In [ ]:
fig, ax = ant.map_grid(background, return_fig_ax=True, coastline_res='10m', figsize = (16,16))

fig_tam, ax_tam = ant.map_grid(background, 
                       return_fig_ax=True, 
                       coastline_res='10m',  
                       extent=tam_extent, 
                        figsize=tam_figsize
                       )

ax.plot(x_vert_tam,y_vert_tam, transform=ccrs.SouthPolarStereo(), 
       zorder =20, c = 'k', alpha = 0.7, linestyle='--')

cooling_mask = df['age_significance'] == 'Cooling'
for chron_type in cooling_isotopes[::-1]:
    mask = np.logical_and(df['mineral_isotope'] == chron_type, cooling_mask)
    
    for a in [ax, ax_tam]: # 
        a.scatter(df['longitude'][mask], 
                df['latitude'][mask], 
                c = df['age_ma'][mask],
               cmap = cmap_strat,
                norm = norm_strat,
                s = ms*1.2,
                label = label,
                linewidths = 0.9,
                edgecolors = 'k',
                marker = chron_dict[chron_type],
                zorder = 20, 
                alpha = 1,
                transform=ccrs.PlateCarree(), )
    

fig.savefig('fig/fig_4c.pdf', bbox_inches='tight', pad_inches=0)

fig_tam.savefig('fig/fig_4c_tam.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
fig, ax = ant.map_grid(background, return_fig_ax=True, coastline_res='10m', figsize = (16,16))




scatter = scatter_data_map('density_model',
                cmap = cm.batlow_r,
                vmin =2400,
                vmax = 3400,
                ax=ax)



fig.savefig('fig/fig_5a.pdf')


In [ ]:
make_legend([2600, 2800, 3000, 3200, 3400], 'fig/fig_5a_legend.pdf', 
            title = 'density \nN = %s\n(kg/m$^3$)'%df['density_model'].count(),
            cmap=cm.batlow_r, vmin = 2400, vmax = 3400)

In [ ]:
# N bins in histograms
bins = 101

In [ ]:
fig, ax = plt.subplots(2,1, sharex=True, figsize = (6, 6))

range_density = (2500, 3600)

bin_offset = (range_density[1]-range_density[0])/bins


for a, data, color in zip(ax, [gard, df], ['#a6cee3', '#b2df8a']):
    
    n, binned, _ = a.hist(data['density_model'], bins = bins, color = color, 
                              histtype = 'step', fill = True,
                             density=True, range=range_density)
    mean_density = data['density_model'].mean()
    max_n = n.max()

    arg_local_max = argrelmax(n, order=20)

    maxs_x = np.take(binned, arg_local_max).flatten()[:2]+bin_offset
    maxs_y = np.take(n, arg_local_max).flatten()[:2]
    
    print(maxs_y)
    
    for max_x, max_y in zip(maxs_x, maxs_y):
        a.text(max_x, max_y*1.05, '%.0f kg/m$^3$'%max_x, color = 'k', rotation=0)
    

    a.set_yticks([])
    a.spines['right'].set_visible(False)
    a.spines['top'].set_visible(False)
    a.set_xlim(2500, 3600)
    
    

    
ax[0].set_ylabel('global\n(freq.)', size=11)
ax[1].set_ylabel('PetroChron\n(freq.)', size=11)

ax[1].set_xlabel('(kg/m$^3$)')
    
fig.savefig('fig/fig_5b.pdf')

In [ ]:
fig, ax = ant.map_grid(background, return_fig_ax=True, coastline_res='10m', figsize = (16,16))



scatter = scatter_data_map('p_velocity',
                cmap = cm.imola_r,
                vmin =6,
                vmax = 8,
                ax=ax)




fig.savefig('fig/fig_5c.pdf')
   

In [ ]:
make_legend([6, 6.5, 7, 7.5, 8], 'fig/fig_5c_legend.pdf', 
            title = 'p-velocity \nN = %s\n(km/s)'%df['p_velocity'].count(),
            cmap=cm.imola_r, vmin = 6, vmax = 8)

In [ ]:
fig, ax = plt.subplots(2,1, sharex=True, figsize = (6, 6))


range_velocity = (5.5, 8.5)

bin_offset = (range_velocity[1]-range_velocity[0])/bins/2

for a, data, color, label in zip(ax, [gard, df], ['#a6cee3', '#b2df8a'], ['Global (total)', 'PetroChron (total)']):
    
    n, binned, _ = a.hist(data['p_velocity'], bins = bins, 
                             color = color, 
                            histtype = 'step', 
                             fill = True, 
                             label='Total',
                             density=True, 
                             range=range_velocity)
    mean_density = data['p_velocity'].mean()
    max_n = n.max()

    arg_local_max = argrelmax(n, order=4)

    maxs_x = np.take(binned, arg_local_max).flatten()[:2] + bin_offset 
    
    maxs_y = np.take(n, arg_local_max).flatten()[:2]
    
    
    #a.vlines(maxs_x, ymin = 0, ymax=maxs_y, color='k', alpha =0.5)  
    
    for max_x, max_y in zip(maxs_x, maxs_y):
        a.text(max_x, max_y*1.05, '%.1f m/s'%max_x, color = 'k', rotation= 0)

    a.set_yticks([])
    a.spines['right'].set_visible(False)
    a.spines['top'].set_visible(False)
    a.set_xlim(5.5, 8.5)


ax[0].set_ylabel('global\n(freq.)', size=11)
ax[1].set_ylabel('PetroChron\n(freq.)', size=11)

ax[1].set_xlabel('(m/s)')
    
fig.savefig('fig/fig_5d.pdf')

In [ ]:
fig, ax = ant.map_grid(background, return_fig_ax=True, coastline_res='10m', figsize = (16,16))



scatter = scatter_data_map('heat_production',
                cmap = cm.lajolla_r,
                sort=True,
                vmin = 0,
                vmax = 14,
                ax=ax)


fig.savefig('fig/fig_5e.pdf')
   


In [ ]:
make_legend([0,2,4,6,8,10,12, 14], 'fig/fig_5e_legend.pdf', 
            title = 'heat production \nN = %s \n($\mu$W/m$^3$)'%df['heat_production'].count(),      
            cmap=cm.lajolla_r,  ncol = 2, vmin = 0, vmax = 14)

In [ ]:
fig, ax = plt.subplots(2,1, sharex=True, figsize = (6, 6))

ScalarFormatter().set_scientific(False)

for a, data, color in zip(ax, [gard, df], ['#a6cee3', '#b2df8a']):
    
    n, binned, _ = a.hist(data['heat_production'], bins = bins, 
                             color = color, fill = True, density=True, 
                             range=(0, 10), histtype = 'step',)
    mean_density = data['heat_production'].mean()
    median_density = data['heat_production'].median()
    max_n = n.max()
    
    
    p25 = np.nanpercentile(data['heat_production'], 25)
    p75 = np.nanpercentile(data['heat_production'], 75)

    a.vlines(median_density, ymin = 0, ymax=max_n*.9, color='k', alpha =0.5)  
    a.text(median_density, max_n*0.95, 'median:%.1f $\mu$W/m$^3$'%median_density, color = 'k')
    
    a.vlines(p25, ymin = 0, ymax=max_n*1.25, color='k', alpha =0.5)  
    a.text(p25, max_n*1.3, '25th percentile:%.1f $\mu$W/m$^3$'%p25, color = 'k')
    
    a.vlines(p75, ymin = 0, ymax=max_n*0.45, color='k', alpha =0.5)  
    a.text(p75, max_n*0.5, '75th percentile:%.1f $\mu$W/m$^3$'%p75, color = 'k')
    

    a.set_yticks([])
    a.spines['right'].set_visible(False)
    a.spines['top'].set_visible(False)
    a.set_xlim(0.0, 7)
    
ax[0].set_ylabel('global\n(freq.)', size=11)
ax[1].set_ylabel('PetroChron\n(freq.)', size=11)

ax[1].set_xlabel('$\mu$W/m$^3$')
    
fig.savefig('fig/fig_5f.pdf')

In [ ]:
# Compile figures using LaTeX and TikZ, also make small jpg versions

In [ ]:
! pdflatex --interaction=batchmode Fig_1.tex 2>&1 > /dev/null
! convert -density 300 Fig_1.pdf -background White -alpha Background -quality 80 export_jpg/Fig_1.jpg

In [ ]:
! pdflatex --interaction=batchmode Fig_3.tex 2>&1 > /dev/null
! convert -density 300 Fig_3.pdf -background White -alpha Background -quality 80 export_jpg/Fig_3.jpg

In [ ]:
! pdflatex --interaction=batchmode Fig_4.tex 2>&1 > /dev/null
! convert -density 300 Fig_4.pdf -background White -alpha Background -quality 80 export_jpg/Fig_4.jpg

In [ ]:
! pdflatex --interaction=batchmode Fig_5.tex 2>&1 > /dev/null
! convert -density 300 Fig_5.pdf -background White -alpha Background -quality 80 export_jpg/Fig_5.jpg

In [ ]:
(df['heat_production'] <= 0.01).sum()